# Comandos para realização do trabalho da matéria de Big Data com uso da biblioteca PySpark.

Este notebook foi projetado para guiar os alunos na realização das práticas de Big Data utilizando PySpark. Certifique-se de seguir cada etapa cuidadosamente para garantir a correta execução das atividades.

Seu trabalho começará na célula 5. Execute as 4 primeiras células para iniciar a atividade.

## <font color=red>Observação importante:</font>

<font color=yellow>Trabalho realizado com uso da biblioteca pandas não será aceito!</font>

## Upload do arquivo `imdb-reviews-pt-br.csv` para dentro do Google Colab

Aqui, você fará o download do dataset necessário para as atividades. Certifique-se de que o arquivo foi descompactado corretamente antes de prosseguir.

In [5]:
import os
import requests
import zipfile

url = "https://raw.githubusercontent.com/N-CPUninter/Big_Data/main/data/imdb-reviews-pt-br.zip"
filename = "imdb-reviews-pt-br.zip"

response = requests.get(url, stream=True)

if response.status_code == 200:
    with open(filename, "wb") as f:
        for chunk in response.iter_content(1024):
            f.write(chunk)
    print(f"Downloaded {filename} successfully!")
else:
    print(f"Error downloading file: {response.status_code}")

with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall()
    print("Extracted files from", filename)

os.remove(filename)
print(f"Removed {filename}")

Downloaded imdb-reviews-pt-br.zip successfully!
Extracted files from imdb-reviews-pt-br.zip
Removed imdb-reviews-pt-br.zip


## Instalação manual das dependências para uso do pyspark no Google Colab

Esta etapa garante que todas as bibliotecas necessárias para o PySpark sejam instaladas no Google Colab.

In [ ]:
!pip install pyspark

## Importar, instanciar e criar a SparkSession

A SparkSession é o ponto de entrada para usar o PySpark. Certifique-se de configurar corretamente o nome do aplicativo e o master.

In [6]:
from pyspark.sql import SparkSession

MEU_RU = "4467541"
appName = f"PySpark Trabalho de Big Data - {MEU_RU}"
master = "local"

print(f"Iniciando spark session para Ezequiel - RU {MEU_RU}")
spark: SparkSession = SparkSession.builder.appName(appName).master(master).getOrCreate()
print("Sessão iniciado com sucesso! 🚀")

Iniciando spark session para Ezequiel - RU 4467541
Sessão iniciado com sucesso! 🚀


## Criar spark dataframe do CSV utilizando o método read.csv do spark

Não altere este código e use o dataframe imdb_df criado aqui em todo o seu trabalho. A criação de um dataframe diferente deste poderá causar erros na coluna sentiment e isso refletirá em erros de resposta das questões.

In [7]:
from pyspark.sql import DataFrame
from pyspark.sql.types import StructType, StructField, StringType

schema = StructType([
  StructField("id", StringType(), True),
  StructField("text_en", StringType(), True),
  StructField("text_pt", StringType(), True),
  StructField("sentiment", StringType(), True),
])


imdb_df: DataFrame = spark.read.csv('imdb-reviews-pt-br.csv',
                         header=True,
                         quote="\"",
                         escape="\"",
                         encoding="UTF-8")

# Questão 1

Nesta questão, você irá calcular a soma dos IDs para entradas onde o sentimento ('sentiment') é 'neg'.

### Objetivo:
- Usar a coluna 'sentiment' como chave e somar os valores da coluna 'id'.

## Criar funções de MAP:
- Criar função para mapear o "sentiment" como chave e o "id" como valor do tipo inteiro

A função map irá transformar cada linha do dataframe em uma **tupla** (chave-valor), onde:
- Chave: coluna 'sentiment'
- Valor: coluna 'id' convertida para inteiro.

In [13]:
from pyspark.sql import DataFrame

def filter_negative_reviews(data: DataFrame) -> DataFrame:

    return data.filter(data["sentiment"] == "neg")

## Cria funções de REDUCE:

- Criar função de reduce para somar os IDs por "sentiment".

A função reduce irá somar os valores dos IDs agrupados por chave ('sentiment').

In [17]:
from pyspark.sql import DataFrame
from pyspark.sql.functions import col, sum

def sum_negative_ids(reviews: DataFrame) -> DataFrame:

  return reviews.withColumn("id", col("id").cast("int")).select(sum("id"))

## Aplicação do map/reduce e visualização do resultado

Aqui, você aplicará as funções de map e reduce ao dataframe Spark para calcular os resultados. Não se esqueça de usar o método `.collect()` para visualizar os resultados.

In [23]:
negative_reviews = filter_negative_reviews(imdb_df)
sum_of_negative_ids = sum_negative_ids(negative_reviews).collect()[0][0]

print(f"Meu nome é Ezequiel é meu RU é 4467541")
print(f"Soma de IDs das reviews negativas: {sum_of_negative_ids}")

Meu nome é Ezequiel é meu RU é 4467541
Soma de IDs das reviews negativas: 459568555


# Questão 2:

Nesta questão, você irá calcular a diferença no número total de palavras entre textos negativos em português e inglês.

### Objetivo:
- Contar as palavras em cada idioma (colunas 'text_pt' e 'text_en') para entradas onde o sentimento ('sentiment') é 'neg'.
- Subtrair o total de palavras em inglês do total em português.

## Criar funções de MAP:
- Criar função para mapear o "sentiment" como chave de uma tupla principal e como valor uma outra tupla com a soma das palavras de cada idioma como valor.

A função map irá transformar cada linha do dataframe em uma tupla (chave-valor), onde:
- Chave: coluna 'sentiment'
- Valor: Nova tupla com:
  - Elemento 0: soma das palavras da coluna 'text_en'
  - Elemento 1: soma das palavras da coluna 'text_pt'

OU
- Chave: coluna 'sentiment'
- Valor: (soma das palavras da coluna 'text_pt') - (soma das palavras da coluna 'text_en')
  

Para contar as palavras deve-se primeiro separar os textos em uma lista de palavras para então descobrir o tamanho desta lista.
Dicas:

1. Use o método .split() e não .split(" ") de string para separar as palavras em uma lista ou use a função split(coluna de texto, regex) do pyspark com o regex igual à "[ ]+" ou "\s+"
2. Use len() para descobrir o tamanho da lista de palavras.

In [19]:
from pyspark.sql import DataFrame
from pyspark.sql.functions import col, split, size

def map_sentiment_to_word_count(data: DataFrame) -> DataFrame:

  return data.select(col("sentiment"), size(split(col("text_en"), "\\s+")).alias("text_en_word_count"), size(split(col("text_pt"), "\\s+")).alias("text_pt_word_count"))

## Cria funções de REDUCE:

- Criar função de reduce para somar o numero de palavras de cada texto português e inglês por "sentiment" (dependerá de como você optou por fazer sua função map2).

A função reduce irá somar os valores das quantidades de palavras agrupados por chave ('sentiment').

In [20]:
from pyspark.sql import DataFrame

def reduce_word_count_by_sentiment(sentiment_word_counts: DataFrame) -> DataFrame:

  return sentiment_word_counts.groupBy("sentiment").agg(sum("text_en_word_count").alias("total_text_en_words"), sum("text_pt_word_count").alias("total_text_pt_words"))


## Aplicação do map/reduce e visualização do resultado

1. Aplicar o map/reduce no seu dataframe spark e realizar o collect() ao final
2. Selecionar os dados referentes aos textos negativos para realizar a subtração.
3. Realizar a subtração das contagens de palavras dos textos negativos para obter o resultado final

In [22]:
negative_data = filter_negative_reviews(imdb_df)

sentiment_word_counts = map_sentiment_to_word_count(negative_data)
total_word_counts = reduce_word_count_by_sentiment(sentiment_word_counts)
result = total_word_counts.collect()[0]
word_count_difference = result[2] - result[1]

print(f"Meu nome é Ezequiel é meu RU é 4467541")
print(f"Diferença entre a contagem de palavras: {word_count_difference} (Texto em PT - Texto em EN)")


Meu nome é Ezequiel é meu RU é 4467541
Diferença entre a contagem de palavras: 54976 (Texto em PT - Texto em EN)
